### Import Dependencies

In [1]:
import pandas as pd
import numpy as np
import os
import re
import matplotlib as plt

### Set Formulas

In [2]:
def tallies(df):
    df['PF'].fillna(0, inplace = True)
    df['PA'].fillna(0, inplace = True)
    df['OPassY'].fillna(0, inplace = True)
    df['ORushY'].fillna(0, inplace = True)
    df['TO_lost'].fillna(0, inplace = True)
    df['DPassY'].fillna(0, inplace = True)
    df['DRushY'].fillna(0, inplace = True)
    df['TO_won'].fillna(0, inplace = True)
    df['OTotYd'].fillna(0, inplace = True)
    df['DTotYd'].fillna(0, inplace = True)

    df['PF Tally'] = df['PF'].cumsum() - df['PF']
    df['PA Tally'] = df['PA'].cumsum() - df['PA']
    df['OPassY Tally'] = df['OPassY'].cumsum() - df['OPassY']
    df['ORushY Tally'] = df['ORushY'].cumsum() - df['ORushY']
    df['TO_lost Tally'] = df['TO_lost'].cumsum() - df['TO_lost']
    df['TO_won Tally'] = df['TO_won'].cumsum() - df['TO_won']
    df['DPassY Tally'] = df['DPassY'].cumsum() - df['DPassY']
    df['DRushY Tally'] = df['DRushY'].cumsum() - df['DRushY']
    df['OTotYd Tally'] = df['OTotYd'].cumsum() - df['OTotYd']
    df['DTotYd Tally'] = df['DTotYd'].cumsum() - df['DTotYd']

    df = df.drop(columns=['PF', 'PA', 'OPassY', 'ORushY', 'TO_lost', 'TO_won', 'DPassY', 'DRushY', 'OTotYd', 'DTotYd'])
    
    return df

In [6]:
def clean_columns(df, home_team=''):
    # Delete 'Bye' weeks and change column name
    df = df[df['Opp'] != 'Bye Week']
    df = df.rename(columns={"Entering Wins": "Wins Tally"})

    # Clean 'Home' column so home games have a value of 1 and away games have a value of 0
    df['Home'] = df['Home'].replace(['@', 'N'], 0)
    df['Home'].fillna(1, inplace = True)

    # Clean 'Results' column so wins have a value of 1 and losses/ties have a value of 0
    df['Results'] = df['Results'].replace(['L', 'T'], 0)
    df['Results'] = df['Results'].replace('W', 1)

    # Drop and add columns as needed
    df = df.drop(columns=['Rec', 'Date'])
    df['Home_Team'] = home_team
    
    return df

In [7]:
def win_rate_by_record(team_df, team, wins, week):
    df = team_df[team_df['Home_Team'] == team]
    df = df[df['Entering Wins'] >= wins]
    df = df[df['Week'] == week]
    wins_df = df[df['Results'] == 1]
    if len(df) == 0:
        return f'No data.'
    else:
        win_pt = float("{:.2f}".format(len(wins_df) / len(df) * 100))
        return win_pt

In [8]:
def win_rate_by_pf(team_df, team, pf):
    df = team_df[team_df['Home_Team'] == team]
    df = df[df['PF'] >= pf]
    wins_df = df[df['Results'] == 1]
    if len(df) == 0:
        return f'No data.'
    else:
        win_pt = float("{:.2f}".format(len(wins_df) / len(df) * 100))
        return win_pt

In [9]:
def win_rate_by_pa(team_df, team, pa):
    df = team_df[team_df['Home_Team'] == team]
    df = df[df['PA'] >= pa]
    wins_df = df[df['Results'] == 1]
    if len(df) == 0:
        return f'No data.'
    else:
        win_pt = float("{:.2f}".format(len(wins_df) / len(df) * 100))
        return win_pt

In [10]:
def win_rate_by_opp(team_df, team, opp):
    df = team_df[team_df['Home_Team'] == team]
    df = df[df['Opp'] == opp]
    wins_df = df[df['Results'] == 1]
    if len(df) == 0:
        return f'No data.'
    else:
        win_pt = float("{:.2f}".format(len(wins_df) / len(df) * 100))
        return win_pt

In [11]:
def avg_pf_by_team(team_df, team, opp):
    df = team_df[team_df['Home_Team'] == team]
    df = df[df['Opp'] == opp]
    pf_avg = df['PF'].mean()
    pf_avg_rounddown = pf_avg - (pf_avg % 1)
    return pf_avg_rounddown

In [12]:
def avg_pa_by_team(team_df, team, opp):
    df = team_df[team_df['Home_Team'] == team]
    df = df[df['Opp'] == opp]
    pa_avg = df['PA'].mean()
    pa_avg_roundup = pa_avg + (1 - (pa_avg % 1))
    return pa_avg_roundup

In [13]:
def total_avg(team_df, team, wins, week, opp, pa, pf):
        
    df = team_df[team_df['Home_Team'] == team]
    percentage_sum = 0
    count = 0

    # win_rate_by_ record
    df = team_df[team_df['Home_Team'] == team]
    df = df[df['Entering Wins'] >= wins]
    df = df[df['Week'] == week]
    wins_df = df[df['Results'] == 1]
    if len(df) == 0:
        pass
    else:
        win_pt1 = float("{:.2f}".format(len(wins_df) / len(df) * 100))
        percentage_sum = percentage_sum + win_pt1
        count = count + 1

    # win_rate_by_ opp
    df = team_df[team_df['Home_Team'] == team]
    df = df[df['Opp'] == opp]
    wins_df = df[df['Results'] == 1]
    if len(df) == 0:
        pass
    else:
        win_pt2 = float("{:.2f}".format(len(wins_df) / len(df) * 100))
        percentage_sum = percentage_sum + win_pt2
        count = count + 1

    # win_rate_by_ avg pf
    df = team_df[team_df['Home_Team'] == team]
    df = df[df['PF'] >= pf]
    wins_df = df[df['Results'] == 1]
    if len(df) == 0:
        pass
    else:
        win_pt5 = win_rate_by_pf(team_df=team_df, team=team, pf=avg_pf_by_team(team_df=team_df, team=team, opp=opp))
        percentage_sum = percentage_sum + win_pt5
        count = count + 1

    # win_rate_by_ avg pa
    df = team_df[team_df['Home_Team'] == team]
    df = df[df['PA'] >= pa]
    wins_df = df[df['Results'] == 1]
    if len(df) == 0:
        pass
    else:
        win_pt6 = win_rate_by_pa(team_df=team_df, team=team, pa=avg_pa_by_team(team_df=team_df, team=team, opp=opp))
        percentage_sum = percentage_sum + win_pt6
        count = count + 1

    total_avg = float("{:.2f}".format(percentage_sum/count))
    return total_avg

In [14]:
def odds_summary(team_df, team, opp, week, wins=0, pf=0, pa=0):    
    print(f'Avg Win Rating - Overall: {total_avg(team_df=team_df, team=team, wins=wins, week=week, opp=opp, pa=pa, pf=pf)}%')
    print(f'-----')
    print(f'Against {opp}, Win Rate: {win_rate_by_opp(team_df=team_df, team=team, opp=opp)}%')
    print(f'Week {week}- Wins by: {wins}, Win Rate: {win_rate_by_record(team_df=team_df, team=team, wins=wins, week=week)}%')
    print(f'Avg PF: {avg_pf_by_team(team_df=team_df, team=team, opp=opp)}, Win Rate: {win_rate_by_pf(team_df=team_df, team=team, pf=avg_pf_by_team(team_df=team_df, team=team, opp=opp))}%')
    print(f'Avg PA: {avg_pa_by_team(team_df=team_df, team=team, opp=opp)}, Win Rate: {win_rate_by_pa(team_df=team_df, team=team, pa=avg_pa_by_team(team_df=team_df, team=team, opp=opp))}%')

In [15]:
# Generate our categorical variable list
def cat_columns(df):
    cats = df.dtypes[df.dtypes == "object"].index.tolist()
    return cats

In [16]:
# Get unique values in each column with categorical values
def vcount_summary(df, cat_list):
    for i in cat_list:
        print(f'Column: {i}')
        print(f'Unique Values: {df[i].nunique()}')
        print(df[i].value_counts())
        print('-------------')

In [17]:
# Make function to bucket low values into 'other' type
def other_bucket(df, column, cutoff):
    counts = df[column].value_counts()
    replacements = list(counts[counts < cutoff].index)
    
    for i in replacements:
        df[column] = df[column].replace(i, 'Other')

In [18]:
# Encode categorical columns and merge with primary dataframe
def encode_merge(df, cat_list):
    for i in cat_list:
        encode_df = pd.DataFrame(enc.fit_transform(df[i].values.reshape(-1,1)))
        encode_df.columns = enc.get_feature_names([i])
        df = df.merge(encode_df,left_index=True,right_index=True).drop(i,1)
    return df

In [24]:
def yearly_data(df, year, HC2_range=0, QB2_range=0, HC='', HC2='none', OC='', DC='', QB='', QB2='none'):
    df['year'] = year
    df = tallies(df)
    df['Head_Coach'] = HC
    if HC2 == 'none':
        pass
    else:
        df.loc[df['Week'] == HC2_range, 'Head_Coach'] = HC2
    df['OC'] = OC
    df['DC'] = DC
    df['QB'] = QB
    if QB2 == 'none':
        pass
    else:
        df.loc[df['Week'] == QB2_range, 'Head_Coach'] = QB2
        
    return df

### San Francisco 49ers Data, 2010-2019

In [25]:
sf_df2010 = pd.read_csv('Resources/sf_sportsref_download2010.csv')
sf_df2011 = pd.read_csv('Resources/sf_sportsref_download2011.csv')
sf_df2012 = pd.read_csv('Resources/sf_sportsref_download2012.csv')
sf_df2013 = pd.read_csv('Resources/sf_sportsref_download2013.csv')
sf_df2014 = pd.read_csv('Resources/sf_sportsref_download2014.csv')
sf_df2015 = pd.read_csv('Resources/sf_sportsref_download2015.csv')
sf_df2016 = pd.read_csv('Resources/sf_sportsref_download2016.csv')
sf_df2017 = pd.read_csv('Resources/sf_sportsref_download2017.csv')
sf_df2018 = pd.read_csv('Resources/sf_sportsref_download2018.csv')
sf_df2019 = pd.read_csv('Resources/sf_sportsref_download2019.csv')

sf_df2010 = yearly_data(df=sf_df2010, year=2010, 
                        HC='Mike Singletary', HC2_range=17, HC2='Jim Tomsula', 
                        OC='Jimmy Raye', 
                        DC='Greg Manusky', 
                        QB='Alex Smith', 
                        QB2='David Carr', QB2_range=7)
sf_df2010.loc[sf_df2010['QB'] == range(8,13,1), 'QB'] = "Troy Smith"

sf_df2011 = yearly_data(df=sf_df2011, year=2011, 
                        HC='Jim Harbaugh', 
                        OC='Greg Roman', 
                        DC='Vic Fangio', 
                        QB='Alex Smith')
sf_df2012 = yearly_data(df=sf_df2012, year=2012, 
                        HC='Jim Harbaugh', 
                        OC='Greg Roman', 
                        DC='Vic Fangio',
                        QB='Alex Smith', 
                        QB2='Colin Kaepernick', QB2_range=range(10,18,1))
sf_df2013 = yearly_data(df=sf_df2013, year=2013, 
                        HC='Jim Harbaugh', 
                        OC='Greg Roman', 
                        DC='Vic Fangio', 
                        QB='Colin Kaepernick')
sf_df2014 = yearly_data(df=sf_df2014, year=2014, 
                        HC='Jim Harbaugh', 
                        OC='Greg Roman', 
                        DC='Vic Fangio', 
                        QB='Colin Kaepernick')
sf_df2015 = yearly_data(df=sf_df2015, year=2015, 
                        HC='Jim Tomsula', 
                        OC='Geep Chryst', 
                        DC='Eric Mangini', 
                        QB='Colin Kaepernick',
                        QB2='Blaine Gabbert', QB2_range=range(10,18,1))
sf_df2016 = yearly_data(df=sf_df2016, year=2016, 
                        HC='Chip Kelly', 
                        OC='Curtis Modkins', 
                        DC="Jim O'Neil", 
                        QB='Blaine Gabbert',
                        QB2='Colin Kaepernick', QB2_range=(6,18,1))
sf_df2017 = yearly_data(df=sf_df2017, year=2017, 
                        HC='Kyle Shanahan', 
                        OC='Kyle Shanahan', 
                        DC='Robert Saleh', 
                        QB='Brian Hoyer',
                        QB2='C.J. Beathard', QB2_range=range(7,13,1))
sf_df2017.loc[sf_df2017['QB'] == range(12,18,1), 'QB'] = "Jimmy Garoppolo"

sf_df2018 = yearly_data(df=sf_df2018, year=2018, 
                        HC='Kyle Shanahan', 
                        OC='Kyle Shanahan', 
                        DC='Robert Saleh', 
                        QB='Jimmy Garoppolo',
                        QB2='C.J. Beathard', QB2_range=range(3,9,1))
sf_df2018.loc[sf_df2018['QB'] == range(8,18,1), 'QB'] = "Nick Mullens"

sf_df2019 = yearly_data(df=sf_df2019, year=2019, 
                        HC='Kyle Shanahan', 
                        OC='Kyle Shanahan', 
                        DC='Robert Saleh', 
                        QB='Jimmy Garoppolo')

sf_df = sf_df2019.append(sf_df2018, ignore_index=True)
sf_df = sf_df.append(sf_df2017, ignore_index=True)
sf_df = sf_df.append(sf_df2016, ignore_index=True)
sf_df = sf_df.append(sf_df2015, ignore_index=True)
sf_df = sf_df.append(sf_df2014, ignore_index=True)
sf_df = sf_df.append(sf_df2013, ignore_index=True)
sf_df = sf_df.append(sf_df2012, ignore_index=True)
sf_df = sf_df.append(sf_df2011, ignore_index=True)
sf_df = sf_df.append(sf_df2010, ignore_index=True)

sf_df = clean_columns(df=sf_df, home_team='49ers')

sf_df

,Week,Day,Results,Wins Tally,Home,Opp,week_after_bye,year,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,Head_Coach,OC,DC,QB,Home_Team
0,1.0,Sun,1.0,0.0,0.0,Tampa Bay Buccaneers,0.0,2019,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Kyle Shanahan,Kyle Shanahan,Robert Saleh,Jimmy Garoppolo,49ers
1,2.0,Sun,1.0,1.0,0.0,Cincinnati Bengals,0.0,2019,31.0,17.0,...,4.0,174.0,121.0,256.0,295.0,Kyle Shanahan,Kyle Shanahan,Robert Saleh,Jimmy Garoppolo,49ers
2,3.0,Sun,1.0,2.0,1.0,Pittsburgh Steelers,0.0,2019,72.0,34.0,...,5.0,465.0,146.0,827.0,611.0,Kyle Shanahan,Kyle Shanahan,Robert Saleh,Jimmy Garoppolo,49ers
4,5.0,Mon,1.0,3.0,1.0,Cleveland Browns,1.0,2019,96.0,54.0,...,7.0,625.0,225.0,1263.0,850.0,Kyle Shanahan,Kyle Shanahan,Robert Saleh,Jimmy Garoppolo,49ers
5,6.0,Sun,1.0,4.0,0.0,Los Angeles Rams,0.0,2019,127.0,57.0,...,11.0,703.0,327.0,1709.0,1030.0,Kyle Shanahan,Kyle Shanahan,Robert Saleh,Jimmy Garoppolo,49ers
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,13.0,Sun,0.0,4.0,0.0,Green Bay Packers,0.0,2010,187.0,225.0,...,14.0,2404.0,1081.0,3523.0,3485.0,Mike Singletary,Jimmy Raye,Greg Manusky,Alex Smith,49ers
179,14.0,Sun,1.0,4.0,1.0,Seattle Seahawks,0.0,2010,203.0,259.0,...,14.0,2678.0,1217.0,3792.0,3895.0,Mike Singletary,Jimmy Raye,Greg Manusky,Alex Smith,49ers
180,15.0,Thu,0.0,5.0,0.0,San Diego Chargers,0.0,2010,243.0,280.0,...,19.0,2955.0,1301.0,4128.0,4256.0,Mike Singletary,Jimmy Raye,Greg Manusky,Alex Smith,49ers
181,16.0,Sun,0.0,5.0,0.0,St. Louis Rams,0.0,2010,250.0,314.0,...,19.0,3221.0,1409.0,4320.0,4630.0,Mike Singletary,Jimmy Raye,Greg Manusky,Alex Smith,49ers


In [26]:
niners_df = sf_df.copy()
niners_df.to_csv('Resources/niners.csv')

### Rams Data, 2010-2019

In [ ]:
# Data Source: https://www.pro-football-reference.com/teams/sfo/index.htm
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

rams_df2010 = pd.read_csv('Resources/rams_sportsref_download2010.csv')
rams_df2011 = pd.read_csv('Resources/rams_sportsref_download2011.csv')
rams_df2012 = pd.read_csv('Resources/rams_sportsref_download2012.csv')
rams_df2013 = pd.read_csv('Resources/rams_sportsref_download2013.csv')
rams_df2014 = pd.read_csv('Resources/rams_sportsref_download2014.csv')
rams_df2015 = pd.read_csv('Resources/rams_sportsref_download2015.csv')
rams_df2016 = pd.read_csv('Resources/rams_sportsref_download2016.csv')
rams_df2017 = pd.read_csv('Resources/rams_sportsref_download2017.csv')
rams_df2018 = pd.read_csv('Resources/rams_sportsref_download2018.csv')
rams_df2019 = pd.read_csv('Resources/rams_sportsref_download2019.csv')

rams_df2010 = yearly_data(df=rams_df2010, year=2010, 
                        HC='Mike Singletary', HC2_range=17, HC2='Jim Tomsula', 
                        OC='Jimmy Raye', 
                        DC='Greg Manusky', 
                        QB='Alex Smith', 
                        QB2='David Carr', QB2_range=7)
rams_df2010.loc[rams_df2010['QB'] == range(8,13,1), 'QB'] = "Troy Smith"

rams_df2011 = yearly_data(df=rams_df2011, year=2011, 
                        HC='Jim Harbaugh', 
                        OC='Greg Roman', 
                        DC='Vic Fangio', 
                        QB='Alex Smith')
rams_df2012 = yearly_data(df=rams_df2012, year=2012, 
                        HC='Jim Harbaugh', 
                        OC='Greg Roman', 
                        DC='Vic Fangio',
                        QB='Alex Smith', 
                        QB2='Colin Kaepernick', QB2_range=range(10,18,1))
rams_df2013 = yearly_data(df=rams_df2013, year=2013, 
                        HC='Jim Harbaugh', 
                        OC='Greg Roman', 
                        DC='Vic Fangio', 
                        QB='Colin Kaepernick')
rams_df2014 = yearly_data(df=rams_df2014, year=2014, 
                        HC='Jim Harbaugh', 
                        OC='Greg Roman', 
                        DC='Vic Fangio', 
                        QB='Colin Kaepernick')
rams_df2015 = yearly_data(df=rams_df2015, year=2015, 
                        HC='Jim Tomsula', 
                        OC='Geep Chryst', 
                        DC='Eric Mangini', 
                        QB='Colin Kaepernick',
                        QB2='Blaine Gabbert', QB2_range=range(10,18,1))
rams_df2016 = yearly_data(df=rams_df2016, year=2016, 
                        HC='Chip Kelly', 
                        OC='Curtis Modkins', 
                        DC="Jim O'Neil", 
                        QB='Blaine Gabbert',
                        QB2='Colin Kaepernick', QB2_range=(6,18,1))
rams_df2017 = yearly_data(df=rams_df2017, year=2017, 
                        HC='Kyle Shanahan', 
                        OC='Kyle Shanahan', 
                        DC='Robert Saleh', 
                        QB='Brian Hoyer',
                        QB2='C.J. Beathard', QB2_range=range(7,13,1))
rams_df2017.loc[rams_df2017['QB'] == range(12,18,1), 'QB'] = "Jimmy Garoppolo"

rams_df2018 = yearly_data(df=rams_df2018, year=2018, 
                        HC='Kyle Shanahan', 
                        OC='Kyle Shanahan', 
                        DC='Robert Saleh', 
                        QB='Jimmy Garoppolo',
                        QB2='C.J. Beathard', QB2_range=range(3,9,1))
rams_df2018.loc[rams_df2018['QB'] == range(8,18,1), 'QB'] = "Nick Mullens"

rams_df2019 = yearly_data(df=rams_df2019, year=2019, 
                        HC='Kyle Shanahan', 
                        OC='Kyle Shanahan', 
                        DC='Robert Saleh', 
                        QB='Jimmy Garoppolo')

# Make master dataframe for team
rams_df = rams_df2019.append(rams_df2018, ignore_index=True)
rams_df = rams_df.append(rams_df2017, ignore_index=True)
rams_df = rams_df.append(rams_df2016, ignore_index=True)
rams_df = rams_df.append(rams_df2015, ignore_index=True)
rams_df = rams_df.append(rams_df2014, ignore_index=True)
rams_df = rams_df.append(rams_df2013, ignore_index=True)
rams_df = rams_df.append(rams_df2012, ignore_index=True)
rams_df = rams_df.append(rams_df2011, ignore_index=True)
rams_df = rams_df.append(rams_df2010, ignore_index=True)

rams_df = clean_columns(df=rams_df, home_team='Rams')

rams_df

In [ ]:
rams_df.to_csv('Resources/rams.csv')

### Chargers Data, 2010-2019

In [ ]:
# Data Source: https://www.pro-football-reference.com/teams/sfo/index.htm
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

chargers_df2010 = pd.read_csv('Resources/chargers_sportsref_download2010.csv')
chargers_df2010['year'] = 2010
chargers_df2010 = tallies(chargers_df2010)
chargers_df2010['Head_Coach'] = ""
chargers_df2010.loc[chargers_df2010['Week'] == , 'Head_Coach'] = ""
chargers_df2010['OC'] = ""
chargers_df2010['DC'] = ""
chargers_df2010['QB'] = ""
chargers_df2010.loc[chargers_df2010['QB'] == , 'QB'] = ""

chargers_df2010 = pd.read_csv('Resources/chargers_sportsref_download2010.csv')
yearly_data(df=chargers_df2010, year=2010, HC='', HC2_range, HC2='none', OC='', DC='', QB2_range, QB2='')


chargers_df2011 = pd.read_csv('Resources/chargers_sportsref_download2011.csv')
chargers_df2011['year'] = 2011
chargers_df2011 = tallies(chargers_df2011)
chargers_df2011['Head_Coach'] = ""
chargers_df2011.loc[chargers_df2011['Week'] == , 'Head_Coach'] = ""
chargers_df2011['OC'] = ""
chargers_df2011['DC'] = ""
chargers_df2011['QB'] = ""
chargers_df2011.loc[chargers_df2011['QB'] == , 'QB'] = ""

chargers_df2012 = pd.read_csv('Resources/chargers_sportsref_download2012.csv')
chargers_df2012['year'] = 2012
chargers_df2012 = tallies(chargers_df2012)
chargers_df2012['Head_Coach'] = ""
chargers_df2012.loc[chargers_df2012['Week'] == , 'Head_Coach'] = ""
chargers_df2012['OC'] = ""
chargers_df2012['DC'] = ""
chargers_df2012['QB'] = ""
chargers_df2012.loc[chargers_df2012['QB'] == , 'QB'] = ""

chargers_df2013 = pd.read_csv('Resources/chargers_sportsref_download2013.csv')
chargers_df2013['year'] = 2013
chargers_df2013 = tallies(chargers_df2013)
chargers_df2013['Head_Coach'] = ""
chargers_df2013.loc[chargers_df2013['Week'] == , 'Head_Coach'] = ""
chargers_df2013['OC'] = ""
chargers_df2013['DC'] = ""
chargers_df2013['QB'] = ""
chargers_df2013.loc[chargers_df2013['QB'] == , 'QB'] = ""

chargers_df2014 = pd.read_csv('Resources/chargers_sportsref_download2014.csv')
chargers_df2014['year'] = 2014
chargers_df2014 = tallies(chargers_df2014)
chargers_df2014['Head_Coach'] = ""
chargers_df2014.loc[chargers_df2014['Week'] == , 'Head_Coach'] = ""
chargers_df2014['OC'] = ""
chargers_df2014['DC'] = ""
chargers_df2014['QB'] = ""
chargers_df2014.loc[chargers_df2014['QB'] == , 'QB'] = ""

chargers_df2015 = pd.read_csv('Resources/chargers_sportsref_download2015.csv')
chargers_df2015['year'] = 2015
chargers_df2015 = tallies(chargers_df2015)
chargers_df2015['Head_Coach'] = ""
chargers_df2015.loc[chargers_df2015['Week'] == , 'Head_Coach'] = ""
chargers_df2015['OC'] = ""
chargers_df2015['DC'] = ""
chargers_df2015['QB'] = ""
chargers_df2015.loc[chargers_df2015['QB'] == , 'QB'] = ""

chargers_df2016 = pd.read_csv('Resources/chargers_sportsref_download2016.csv')
chargers_df2016['year'] = 2016
chargers_df2016 = tallies(chargers_df2016)
chargers_df2016['Head_Coach'] = ""
chargers_df2016.loc[chargers_df2016['Week'] == , 'Head_Coach'] = ""
chargers_df2016['OC'] = ""
chargers_df2016['DC'] = ""
chargers_df2016['QB'] = ""
chargers_df2016.loc[chargers_df2016['QB'] == , 'QB'] = ""

chargers_df2017 = pd.read_csv('Resources/chargers_sportsref_download2017.csv')
chargers_df2017['year'] = 2017
chargers_df2017 = tallies(chargers_df2017)
chargers_df2017['Head_Coach'] = ""
chargers_df2017.loc[chargers_df2017['Week'] == , 'Head_Coach'] = ""
chargers_df2017['OC'] = ""
chargers_df2017['DC'] = ""
chargers_df2017['QB'] = ""
chargers_df2017.loc[chargers_df2017['QB'] == , 'QB'] = ""

chargers_df2018 = pd.read_csv('Resources/chargers_sportsref_download2018.csv')
chargers_df2018['year'] = 2018
chargers_df2018 = tallies(chargers_df2018)
chargers_df2018['Head_Coach'] = ""
chargers_df2018.loc[chargers_df2018['Week'] == , 'Head_Coach'] = ""
chargers_df2018['OC'] = ""
chargers_df2018['DC'] = ""
chargers_df2018['QB'] = ""
chargers_df2018.loc[chargers_df2018['QB'] == , 'QB'] = ""

chargers_df2019 = pd.read_csv('Resources/chargers_sportsref_download2019.csv')
chargers_df2019['year'] = 2019
chargers_df2019 = tallies(chargers_df2019)
chargers_df2019['Head_Coach'] = ""
chargers_df2019.loc[chargers_df2019['Week'] == , 'Head_Coach'] = ""
chargers_df2019['OC'] = ""
chargers_df2019['DC'] = ""
chargers_df2019['QB'] = ""
chargers_df2019.loc[chargers_df2019['QB'] == , 'QB'] = ""

# Make master dataframe for team
chargers_df = chargers_df2019.append(chargers_df2018, ignore_index=True)
chargers_df = chargers_df.append(chargers_df2017, ignore_index=True)
chargers_df = chargers_df.append(chargers_df2016, ignore_index=True)
chargers_df = chargers_df.append(chargers_df2015, ignore_index=True)
chargers_df = chargers_df.append(chargers_df2014, ignore_index=True)
chargers_df = chargers_df.append(chargers_df2013, ignore_index=True)
chargers_df = chargers_df.append(chargers_df2012, ignore_index=True)
chargers_df = chargers_df.append(chargers_df2011, ignore_index=True)
chargers_df = chargers_df.append(chargers_df2010, ignore_index=True)

chargers_df = clean_columns(df=chargers_df, home_team='Chargers')

### Seahawks Data, 2010-2019

### Cardinals Data, 2010-2019

### Packers Data, 2010-2019

### Saints Data, 2010-2019

### Giants Data, 2010-2019

### Bears Data, 2010-2019

### Panthers Data, 2010-2019

### Bucs Data, 2010-2019

### Falcons Data, 2010-2019

### Washington Data, 2010-2019

### Ravens Data, 2010-2019

### Eagles Data, 2010-2019

### Cowboys Data, 2010-2019

### Chiefs Data, 2010-2019

### Lions Data, 2010-2019

### Vikings Data, 2010-2019

### Bengals Data, 2010-2019

### Raiders Data, 2010-2019

### Steelers Data, 2010-2019

### Browns Data, 2010-2019

### Broncos Data, 2010-2019

### Colts Data, 2010-2019

### Broncos Data, 2010-2019

### Texans Data, 2010-2019

### Jets Data, 2010-2019

### Jaguars Data, 2010-2019

### Dolphins Data, 2010-2019

### Patriots Data, 2010-2019

### Bills Data, 2010-2019

### Titans Data, 2010-2019